In [1]:
import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
pd.set_option('display.max_colwidth', None)

pd.options.display.width = 300

In [3]:
from nltk.tokenize import sent_tokenize, word_tokenize
target = 'question'

In [4]:
def get_message(mes):
    dic = {}
    dic['message'] = mes.text.split('\n\n')[-2]
    dic['message_id'] = mes.get('id').replace('message', '')
    dic['from_name'] = mes.find(attrs={'class':'from_name'})
    reply = mes.find(attrs = {'class': 'reply_to details'})
    
    # print(reply)
    if reply:
        dic['go_to_message_id'] = reply.find('a').get('href').replace('#go_to_message', '')
    return dic

In [5]:
def tokenize(text):
    stop_words = ['за', 'там', 'так', 'то', 'все', 'если', '-',
       ':', 'но', 'как', 'по', 'у', 'есть', 'это', 'я', 'с', 'а', 'не', ')', 'и', '.', 'в', ',']
    tokens = word_tokenize(text.lower(), language="russian")
    tokens = list(filter(lambda x:'посовет' not in x, tokens))
    tokens = list(filter(lambda x: x not in stop_words, tokens))
    
    return list(set(tokens))

In [14]:
# папки с ChatExport
sub_list = []
folders = glob.glob("C:/Users/artem/Downloads/Telegram Desktop/ChatExport/ChatExport*")
files = []
df_reply_mapping = pd.DataFrame()
df_questions = pd.DataFrame()
for chat_name,  folder in enumerate(folders):
    files = glob.glob(f"{folder}/*.html")
    messages_all = []
    for file in tqdm(files):
        with open(file, encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            messages = soup.body.find_all(attrs = {'class':'message'})#.text#.split('\n\n')[-2].lower()
        messages_all+=messages
    # chat_name = soup.find(attrs = {'class', "text bold"}).text
    df = pd.DataFrame(list(map(get_message, messages_all)))
    df["chat_name"] = chat_name
    df['tokens'] = df['message'].apply(tokenize)
    target = 'question'
    target_list = []
    for message in df['message'].values:
        if 'посовет' in message:
            target_list.append(1)
        else:
            target_list.append(0) 
    df[target] = target_list
    
    df_reply_mapping_sub = df[~df['go_to_message_id'].isnull()]
    df_reply_mapping_sub = df_reply_mapping_sub.rename(columns={"go_to_message_id":"message_id", "message_id": "reply_id"})
    df_reply_mapping_sub = pd.merge(df, df_reply_mapping_sub, on=['message_id', 'chat_name'], suffixes=['_question', '_reply'])
    df_reply_mapping_sub = df_reply_mapping_sub[df_reply_mapping_sub['question_question']==1]
    df_reply_mapping_sub['message_question_tokens'] = df_reply_mapping_sub['message_question'].apply(tokenize)
    df_questions_sub = df[df['question']==1]
    df_questions_sub = df_questions_sub[['message','message_id', 'chat_name', 'from_name']]
    df_questions_sub['message_tokens'] = df_questions_sub['message'].apply(tokenize)
    
    df_reply_mapping = df_reply_mapping.append(df_reply_mapping_sub)
    df_questions = df_questions.append(df_questions_sub)

100%|██████████| 31/31 [00:14<00:00,  2.17it/s]
C:\Users\artem\AppData\Local\Temp\ipykernel_19492\441847778.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_reply_mapping = df_reply_mapping.append(df_reply_mapping_sub)
C:\Users\artem\AppData\Local\Temp\ipykernel_19492\441847778.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_questions = df_questions.append(df_questions_sub)
100%|██████████| 56/56 [00:32<00:00,  1.73it/s]
C:\Users\artem\AppData\Local\Temp\ipykernel_19492\441847778.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_reply_mapping = df_reply_mapping.append(df_reply_mapping_sub)
C:\Users\artem\AppData\Local\Temp\ipykernel_19492\441847778.py:38: FutureWarning: The frame.append method is deprec

In [20]:
del df

In [21]:
# df_questions

In [22]:
all_tokens = []
for tokens in df_questions['message_tokens'].values:
    all_tokens += tokens

In [23]:
vc = pd.Series(all_tokens).value_counts()

In [24]:
vc[vc<30]

нужна          29
найти          29
чтоб           29
будет          29
которые        28
               ..
запуталась      1
отправиться     1
10-го           1
🥺буду           1
убеждения       1
Length: 8138, dtype: int64

In [25]:
remove_from_tokens = set(vc[vc>30].index)

In [26]:
df_questions['message_tokens'] = df_questions['message_tokens'].apply(lambda x: [token for token in x if token not in remove_from_tokens])
df_questions = df_questions[df_questions['message_tokens'].apply(len)<10]

In [27]:
# df_reply_mapping

In [28]:
df_reply_mapping['message_question_tokens'] = df_reply_mapping['message_question_tokens'].apply(lambda x: [token for token in x if token not in remove_from_tokens])
df_reply_mapping = df_reply_mapping[df_reply_mapping['message_question_tokens'].apply(len)<10]

In [29]:
df_questions.head(2)

,message,message_id,chat_name,from_name,message_tokens
6507,"\nеще эритрит могу посоветовать. Дынный сахар, натур продукт\n",17786,0,[\nDeleted Account\n ],"[дынный, эритрит, натур, еще, сахар, продукт]"
7894,"посоветуйте плиз тренажорку в цетральном Сочи, желательно в шаговой доступности от ЖД Вокзала\n",19202,0,None,"[жд, вокзала, цетральном, шаговой, доступности, тренажорку]"


In [30]:
df_reply_mapping['message_tokens'] = df_reply_mapping['message_question'].apply(tokenize)

In [31]:
df_reply_mapping['key'] = 0
df_questions['key'] = 0

In [32]:
def intersect(row):
    stop_words = {'посоветуйте', 'пожалуйста', 'хорошего', 'хорошую', 'ребята', 'привет',
                  'здравствуйте', 'добрый', 'день', 'кто', 'может', 'посоветуйте', 'кто', 'может',
                 'хорошего', 'мастера',
                  'кто', 'привет', '!', '?', 'посоветовать', 'всем'
                  , 'может', 'хорошую', 'можете', 'добрый',
                'хорошего', 
                  'день', 'добрый', '!', 'посоветуйте',
                 }
    message_question_tokens = set(row['message_question_tokens']) - stop_words
    message_tokens = set(row['message_tokens']) - stop_words
    len1 = len(message_question_tokens&message_tokens)
    len2 = max(len(message_question_tokens), len(message_tokens))
    if len2==0:
        return 0
    return len1/len2

In [70]:
def cosin(row, col1='message_question_tokens', col2='message_tokens'):
    message_question_tokens = row[col1]
    message_tokens = row[col2]
    question_embedding = np.zeros(300)
    message_embedding = np.zeros(300)
    for token in message_question_tokens:
        if token in data:
            question_embedding+=np.array(data[token])
    for token in message_tokens:
        if token in data:
            message_embedding+=np.array(data[token])
    cos_sim = np.dot(question_embedding , message_embedding)/(np.linalg.norm(question_embedding)*np.linalg.norm(message_embedding))
    return cos_sim

In [71]:
# df_reply_mapping

In [72]:
# message_question_tokens

In [73]:
import numpy as np

In [74]:
from functools import reduce

In [38]:
use_tokens = set()
use_tokens |=reduce(lambda x,y: x|y, df_questions['message_tokens'].apply(set).values)
use_tokens |=reduce(lambda x,y: x|y, df_reply_mapping['tokens_question'].apply(set).values)

In [39]:
import io

def load_vectors(fname, use_tokens):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        if tokens[0] in use_tokens:
            data[tokens[0]] = list(map(float, tokens[1:]))
    return data

data = load_vectors("D:/Downloads/cc.ru.300.vec/cc.ru.300.vec", use_tokens)

2000000it [02:06, 15788.99it/s]


In [40]:
# vc

In [41]:
vc['пожалуйста']

349

In [42]:
df_reply_mapping = df_reply_mapping.drop_duplicates(['message_question', 'message_id'])
df_questions = df_questions.drop_duplicates(['message', 'message_id'])

In [43]:
df_cross = pd.merge(
    df_reply_mapping[['message_question', 'message_id', 'message_question_tokens', 'key', 'from_name_question', 'from_name_reply']],
    df_questions,
    on='key')

df_cross = df_cross[(df_cross['message_id_x']!=df_cross['message_id_y'])&(df_cross['message_question']!=df_cross['message'])]
df_cross['intersect'] = df_cross.apply(intersect,axis=1)

In [44]:
df_cross['cosin'] = df_cross.apply(cosin,axis=1)

C:\Users\artem\AppData\Local\Temp\ipykernel_19492\4236333988.py:12: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = np.dot(question_embedding , message_embedding)/(np.linalg.norm(question_embedding)*np.linalg.norm(message_embedding))


In [45]:
df_cross = df_cross[df_cross['message_question']!=df_cross['message']]
df_cross = df_cross.sort_values('intersect')

In [46]:
# df_cross[df_cross['message_question_tokens'].apply(len)<10]

In [47]:
df_cross[(df_cross['cosin'].notnull())&(df_cross['cosin']<0.90)].sort_values('cosin').iloc[-20:]

,message_question,message_id_x,message_question_tokens,key,from_name_question,from_name_reply,message,message_id_y,chat_name,from_name,message_tokens,intersect,cosin
779617,"Ребят, посоветуйте дайвинг клуб?\n",93986,"[дайвинг, клуб]",0,[\nАнастасия Радостева\n ],[\nArtur Lipatov Loading.....\n ],"Добрый вечер, посоветуйте клуб!!! Всем хорошего время прохождения!!\n",128048,8,[\nAnton \n ],"[клуб, прохождения]",0.500000,0.880271
188470,Добрый вечер! А можете посоветовать детского Лора?\n,209677,"[лора, детского]",0,[\nElena Nazarova\n ],[\nOlya \n ],Добрый день. Какого ЛОРа посоветуете?\n,173698,8,[\nSerggio \n ],"[лора, какого]",0.500000,0.881476
820355,Добрый день. Какого ЛОРа посоветуете?\n,173698,"[лора, какого]",0,[\nSerggio \n ],[\nKristina \n ],Добрый вечер! А можете посоветовать детского Лора?\n,209677,4,[\nElena Nazarova\n ],"[лора, детского]",0.500000,0.881476
700450,"Ребята, очень нужно, посоветуйте подобный чат в Геленджике🙏🙏🙏🙏🙏🙏🙏🙏🙏🙏\n",37642,"[геленджике🙏🙏🙏🙏🙏🙏🙏🙏🙏🙏, чат, подобный]",0,[\nОфигеваю От Хотелок Менеджеров\n ],[\nDasha \n ],"Доброго дня всем, может кто-то посоветует чат для творческих людей\n",232057,4,[\nJulia \n ],"[людей, творческих, чат]",0.333333,0.888917
697666,"Ребят а посоветуйте классные бары в Сочи. Только не клубы а именно бары, может тематические или просто хорошие\n",31066,"[клубы, тематические, классные, бары, именно]",0,[\nМатвей Колтышев\n ],[\nAlyona Kotova\n ],"Всем привет, посоветуйте, пожалуйста, хорошие клубы с электронной музыкой или бары в Сочи.\n",195799,8,[\nATMA \n ],"[клубы, музыкой, бары, электронной]",0.400000,0.889533
833314,"Всем привет, посоветуйте, пожалуйста, хорошие клубы с электронной музыкой или бары в Сочи.\n",195799,"[клубы, музыкой, бары, электронной]",0,[\nATMA \n ],[\nOlga \n ],"Ребят а посоветуйте классные бары в Сочи. Только не клубы а именно бары, может тематические или просто хорошие\n",31066,7,[\nМатвей Колтышев\n ],"[клубы, тематические, классные, бары, именно]",0.400000,0.889533
442534,"Хочется в баню сегодня.. посоветуйте в Эстосадке или на Поляне, пожалуйста\n",122759,"[эстосадке, баню, хочется]",0,[\nWhite Lynx By\n ],[\nВлад \n ],"Всем привет, ребята посоветуйте пожалуйста баню, не дорогую.\n",135563,4,[\nВалерий \n ],"[баню, дорогую]",0.333333,0.892960
6076,"\nДа, мне как раз посоветовал @fedor_anikiev Красный штурм\n",23021,"[красный, @, штурм, fedor_anikiev]",0,[\nRoman Kalinichenko\n ],[\nДаша \n ],"\nМне кажется @aangellife , что я знаю , кому это посоветовать 🤣\n",153883,8,None,"[aangellife, 🤣, @, кому, кажется]",0.200000,0.893744
820647,Добрый день. Какого ЛОРа посоветуете?\n,173698,"[лора, какого]",0,[\nSerggio \n ],[\nKristina \n ],А посоветуйте плиз хорошего Лора в КП ( Адлере/Сочи) ? 🙏\n,92235,4,[\nKate Best\n ],"[лора, адлере/сочи]",0.500000,0.893982
669380,"Добрый вечер, посоветуйте в Адлере молодёжный бар\n",490149,"[молодёжный, бар]",0,[\nNatalya Shpankova\n ],[\nМахран Esquire\n ],Друзья посоветуйте места в Адлере где можно вечер провести? Бар ресторан что бы не скучно было)))\n,12846,8,[\nZhanna \n ],"[провести, ресторан, бар, скучно]",0.250000,0.894832


In [101]:
text = 'посоветуйте автосервис'

tokens = tokenize(text)
tokens = [token for token in tokens if token not in remove_from_tokens]

In [102]:
tokens

['автосервис']

In [103]:
df_reply_mapping['new_tokens'] = len(df_reply_mapping)*[tokens]

In [104]:
# df_reply_mapping

In [105]:
df_reply_mapping['cosin'] = df_reply_mapping.apply(lambda x: cosin(x, 'message_tokens', 'new_tokens'),axis=1)

In [106]:
df_reply_mapping[df_reply_mapping['cosin'].notnull()].sort_values('cosin').tail(10)

,message_question,message_id,from_name_question,go_to_message_id,chat_name,tokens_question,question_question,message_reply,reply_id,from_name_reply,tokens_reply,question_reply,message_question_tokens,message_tokens,key,new_tokens,cosin
70546,Здравствуйте. Не посоветуете хороший ски- сервис в Адлере. Новые лыжи нужно настроить😁\n,138891,[\nИнна \n ],NaN,4,"[адлере, здравствуйте, лыжи, нужно, ски-, новые, хороший, настроить😁, сервис]",1,"\nЗа Кедром(стоматология на Ленина) есть Альпийский клуб, там ски-сервис\n",138946,None,"[клуб, кедром, стоматология, альпийский, (, на, ленина, ски-сервис]",0,"[лыжи, ски-, новые, настроить😁, сервис]","[адлере, здравствуйте, лыжи, нужно, ски-, новые, хороший, настроить😁, сервис]",0,[автосервис],0.395507
30253,"Всем привет , посоветуйте хороший автосервис, течёт масло.\n",263449,[\nGalina \n ],NaN,4,"[течёт, масло, привет, автосервис, всем, хороший]",1,"\nНа плотинной улице, до конца и налево\n",263450,[\nIvan \n ],"[на, налево, конца, до, улице, плотинной]",0,"[течёт, масло, автосервис]","[течёт, масло, привет, автосервис, всем, хороший]",0,[автосервис],0.400109
7513,"Привет ,посоветуйте клуб в Сочи с нормальными ценами.\n",23488,[\nMarina \n ],NaN,7,"[клуб, привет, нормальными, ценами, сочи]",1,\nБестужев\n,23489,[\n🟢 Николай ™ \n ],[бестужев],0,"[клуб, нормальными, ценами]","[клуб, привет, нормальными, ценами, сочи]",0,[автосервис],0.401668
21121,Други посоветуйте клуб в центре Сочи\n,117117,[\nАнтон \n ],NaN,8,"[клуб, центре, сочи, други]",1,\nЦентрал парк. Если дип техно любишь\n,117126,[\nЮлия Измайлова\n ],"[техно, парк, дип, централ, любишь]",0,"[клуб, центре, други]","[клуб, центре, сочи, други]",0,[автосервис],0.414451
11231,"всем привет) посоветуйте нормальную автошколу в Сочи, центральный район))\n",75657,[\nКристина Кадырова\n ],NaN,8,"[привет, автошколу, центральный, сочи, район, всем, нормальную]",1,\nСкиф\n,75660,[\nМаксим Сверчков \n ],[скиф],0,"[автошколу, центральный, район, нормальную]","[привет, автошколу, центральный, сочи, район, всем, нормальную]",0,[автосервис],0.415209
109398,"Добрый вечер, посоветуйте в Адлере молодёжный бар\n",490149,[\nNatalya Shpankova\n ],NaN,6,"[добрый, адлере, молодёжный, вечер, бар]",1,\nМалина\n,490157,[\nМахран Esquire\n ],[малина],0,"[молодёжный, бар]","[добрый, адлере, молодёжный, вечер, бар]",0,[автосервис],0.415677
1952,"Доброе утро! Автолюбители, посоветуйте хорошую СТО в Адлере. Личный авто марки Рено.\n",174233,[\nВлад \n ],NaN,4,"[адлере, хорошую, !, марки, рено, доброе, сто, автолюбители, личный, утро, авто]",1,"\nСТО quattro, на Авиационной. Классные ребята!!!\n",174251,[\nMaXteRoK🔥 \n ],"[quattro, авиационной, классные, на, !, ребята, сто]",0,"[марки, рено, сто, автолюбители, личный, авто]","[адлере, хорошую, !, марки, рено, доброе, сто, автолюбители, личный, утро, авто]",0,[автосервис],0.427461
7720,"Коллеги, посоветуйте проверенный автосервис в Адлере, плииз. В Краснополянском не работает сход-развал(((\n",37014,[\nOleg Mmmm\n ],NaN,3,"[адлере, краснополянском, проверенный, автосервис, коллеги, (, плииз, сход-развал, работает]",1,\nАвтосервисhttp://goldenkeyauto.ru/ идеально\n,37247,[\nVera Levdina\n ],"[//goldenkeyauto.ru/, автосервисhttp, идеально]",0,"[краснополянском, проверенный, автосервис, коллеги, плииз, сход-развал, работает]","[адлере, краснополянском, проверенный, автосервис, коллеги, (, плииз, сход-развал, работает]",0,[автосервис],0.436037
38608,Ребят посоветуйте хороший фитнес клуб в сочи\n,195440,[\nNety Fety\n ],NaN,8,"[клуб, ребят, сочи, хороший, фитнес]",1,\nв кудепсте вроде хороший\n,195441,[\nЯ \n ],"[хороший, кудепсте, вроде]",0,"[клуб, фитнес]","[клуб, ребят, сочи, хороший, фитнес]",0,[автосервис],0.456510
82624,\nТогда и посоветуйте мастера по АКПП 🙂\n,397768,[\nDenis Mulitsa\n ],397764,6,"[🙂, тогда, акпп, мастера]",1,\nСие мне неизвестно\n,397770,[\nGP \n ],"[мне, неизвестно, сие]",0,"[🙂, тогда, акпп]","[🙂, тогда, акпп, мастера]",0,[автосервис],0.463327


In [ ]:
aaa

In [54]:
tokens

['ресторан']

In [ ]:
len(use_tokens)

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
view = df_cross[(df_cross['intersect'].notnull())&(df_cross['from_name_question']!=df_cross['from_name'])].iloc[-140:-70]

In [ ]:
# use_tokens

In [ ]:
# data

In [ ]:
find_token = 'остеопата'

In [ ]:
# data.keys()

In [ ]:
dic = {}
for token, emb in data.items():
    cos_sim = np.dot(data[token] , data[find_token])/(np.linalg.norm(data[token] )*np.linalg.norm(data[find_token]))
    dic[token] = cos_sim

In [ ]:
pd.Series(dic).sort_values().iloc[-20:]

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import cos

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
pairwise_distances(

In [ ]:
np.array(data[token] - data[find_token]

In [ ]:
for use_tokens 

In [ ]:
len(data)

In [ ]:
data['автошколу']

In [ ]:
aaa

In [ ]:
new_questing = 'посоветуйте автошколу в сочи'

In [ ]:
new_tokens = tokenize(new_questing)

In [ ]:
new_tokens

In [ ]:
df_reply_mapping['message_tokens'] = len(df_reply_mapping)*[new_tokens]

In [ ]:
# df_reply_mapping

In [ ]:
df_reply_mapping['score'] = df_reply_mapping.apply(intersect,axis=1)

In [ ]:
df_reply_mapping.sort_values('score').iloc[-10:][['message_question', 'message_reply','message_question_tokens', 'score']]

In [ ]:
df['tokens'] = df['text'].apply(tokenize)

In [ ]:
df = df[df['tokens'].apply(len)>=3]

In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
1

In [ ]:
df_train[df_train['question']==1].sample(n=30)

In [ ]:
train_lenght = len(df_train)

In [ ]:
mean_target = df_train['question'].mean()

In [ ]:
from functools import reduce

In [ ]:
from collections import Counter

In [ ]:
for text in df[df['question']==1]['text'].values:
    pass
    # print(text)
    # print()

In [ ]:
# df['question'].sum()

In [ ]:
from tqdm import tqdm

In [ ]:
total_count = Counter()
for tokens in tqdm(df_train['tokens'].values):
    for token in tokens:
        total_count[token]+=1
    

In [ ]:
# total_count

In [ ]:
target_count = Counter()
for tokens in tqdm( df_train[df_train[target]==1]['tokens'].values):
    for token in tokens:
        target_count[token]+=1

In [ ]:
pd.Series(total_count).sort_values().index[-30:]

In [ ]:
pd.Series(target_count).sort_values()[-30:]

In [ ]:
pd.Series(target_count).sort_values().iloc[-50:].index

In [ ]:
# total_count

In [ ]:
# tokens = df_test['tokens'].iloc[0]

In [ ]:
# pd.Series(target_count.values()).value_counts()

In [ ]:
def predict(tokens):
    score = 0
    for token in tokens:
        p += p*(target_count[token])/(total_count[token]+0.0001)
    return p

In [ ]:
df_train['score'] = df_train['tokens'].apply(predict)

In [ ]:
df_test['score'] = df_test['tokens'].apply(predict)

In [ ]:
# df_test.sort_values('score').iloc[-100:]

In [ ]:
# df_test.sort_values('score')

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(df_train[target], df_train['score'])

In [ ]:
df_test['score'].max()

In [ ]:
vc = df[df[target]==1]['tokens'].apply(len).value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(vc.index, vc.values)
plt.show()

In [ ]:
# рабоатет даже, если нет слова посоветуйте

In [ ]:
df_test[df_test['tokens'].apply(len)>3].sort_values('score').iloc[-50:]

In [ ]:
# топ признаков того, что это вопрос/просьба подсказать
stat = {}
for key, val in target_count.items():
    if val>=3:
        rate = (val)/(total_count[key]+1)
        stat[key] = rate

In [ ]:
stat['smm']

In [ ]:
# pd.Series(target_count).sort_values().iloc[-100:-50]

In [ ]:
pd.Series(stat).sort_values().iloc[-50:]